# Problem Set 3: Merging and regular expressions

**Total points (without extra credit)**: 44

**Background on the policy context**: here, we're going to use two datasets to practice reshaping, merging, and regular expression patterns. Both datasets relate to the broader issue of which employers might be violating the rights of temporary guestworkers granted visas under the H-2A program. Here are some articles about potential exploitation of guestworkers by firms and inequality caused by minimal oversight:

- News media coverage of labor abuses of temporary guestworkers: https://www.buzzfeednews.com/article/kenbensinger/the-pushovers 
- GAO report on labor abuses of temporary guestworkers: https://www.gao.gov/products/gao-15-154

The following datasets are located in `pset3_inputdata` (need to unzip): 

- `jobs`: a dataset of guestworker jobs posted by many employers, some of whom have been debarred (banned) from the program for labor abuses; others not debarred
- `debar`: a dataset of employers who committed violations of labor regulations meant to protect temporary guestworkers 


## Resources from class

- [Lecture](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/06_qss20_w23_mergereshape.pdf) and [activity](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/03_reshaping_merging_solutions.ipynb) on exact merging
- [Lecture](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/07_qss20_w23_regex.pdf) and [activity](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/04_regex_blank.ipynb) on regular expressions (both coming in class on Feb. 1)
- DataCamp modules on both of these

# 0. Load packages & data (1 point total)

In [410]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



## 0.1 Load data (0 points)

Load the following dataset stored in `pset3_inputdata`&mdash;`debar.csv`&mdash;storing it as a dataframe named `debar`. This represents employers temporarily banned from hiring workers.

View the `head()` and columns of this dataframe.

In [411]:
## your code here
from zipfile import ZipFile

#open zipfile
with ZipFile("pset3_inputdata.zip") as myzip:
    data = myzip.open("debar.csv")

#read csv
debar = pd.read_csv(data)

debar.head()


,Name,"City, State",Violation,Duration,Start date,End date
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016


The most notable column names mean as follows:
- `Name`:	Company name of agricultural employer
- `City, State`:	City and state where employer located
- `Violation`:	Type of program violation
- `Start date`:	Start date of debarment (temporary ban)

## 0.2 Inspect data (1 point)

Print the number of rows in `debar` versus the number of unique employer names (`Name`). Is there one row per employer or multiple rows for some employers?

In [412]:
## your code here

len(debar)

debar.Name.nunique()

"there are multiple rows for some employers"

114

98

'there are multiple rows for some employers'

# 1. Reshape data and check duplicates (19 points total)

## 1.1 Make indicator for violation number (2 points)

To make it possible to reshape data, make an indicator for the violation number for each business. The indicator should take the value of `viol` if it's the first row/potential violation, `viol2` if the second row/potential violation, etc.

**Hint:**
- One way to do this is by using an if-else statement to check whether the business name is the same as in the row above (assuming rows are ordered by name). Grouping by employer name and checking the number of unique offenses would also work.

In [434]:
## your code here

#sort Dataframe alphabetically by business
debar = debar.sort_values("Name")

#set all violation numbers to viol
debar["violation_number"] = "viol"

#create necessary parameters for reassignment for loop
count = 1

prevname = "placeholder"

#loop through all names
for ele in debar["Name"]:
    
    #check if name is same as one above (ignores very similar but slightly different business names)
    if ele == prevname:
        
        #gets indices of all repeated businesses
        for n in debar[debar.Name == ele].index.tolist():
            
            #leave first violation number unchanged, since first is "viol" not "viol1"
            if count == 1:
                count = count + 1
                
            #add the number of the violation to the end of violation_number
            else:
                debar.loc[debar.index == n, "violation_number"] = ("viol" + str(count))
                count = count + 1
    
    #assign current business name to prevname
    else:
        prevname = ele
        count = 1
        
debar
        

,Name,"City, State",Violation,Duration,Start date,End date,violation_number,is_repeated
94,69 Farms LLC,"Preston, ID",Failure to respond to audit request,2 years,9/24/2016,9/23/2018,viol,False
97,AB Ranch,"Stephenville, TX",Failure to respond to audit request,2 years,4/28/2017,4/27/2019,viol,False
102,Abe- Nancy Froese,"Seminole, TX",Failure to respond to audit request,2 years,11/23/2017,11/23/2019,viol,False
36,Agecy I LLC,"Idabel, OK",Impeding the Audit Process – Non- Response,2 years,8/27/2015,8/26/2017,viol,False
53,"Agecy II, LLC","Ipswich, SD",Impeding the Audit Process – Non- Response,2 years,3/25/2015,3/24/2017,viol,False
...,...,...,...,...,...,...,...,...
73,Xavier Horne,"Lyons, GA",Non-payment of certification fee,1 year,6/16/2016,6/15/2017,viol,True
89,Xavier Horne,"Lyons, GA",Failure to respond to audit request,2 years,9/27/2017,9/26/2019,viol2,True
88,Yesenia Perez,"Axson, GA",Failure to respond to audit request,2 years,9/27/2017,9/26/2019,viol,False
9,Yolanda Chavez,"Santa Maria, CA",Non Payment,1 year,7/23/2014,7/22/2015,viol,False


## 1.2 Clean up state names (3 points)

Inspect the state names in the business violation data. Which states have names sometimes indicated in long format vs. two-letter abbreviation, e.g. "New Hampshire" vs. "NH"? Which of these may have more than one violation?

**Hint:** 
- One way to do this is to extract state names from the `City, State` column using regex and/or string methods.
- The simplest way to check if a state may have more than one violation is to check how many times they appear using `value_counts()`. 

In [435]:
## your code here

#regex to find abbreviated state names
regex_abbrev = ", \w\w'"

#regex to find full state names
regex_full = ", \w\w\w+?'"

#make into sets to erase duplicates, strip all unnecessary characters leftover from regex
abbrev_states = set(re.findall(regex_abbrev, str(debar["City, State"].tolist())))
fullname_states = set(re.findall(regex_full, str(debar["City, State"].tolist())))
abbrev_states = [ele.strip("'").strip('"').strip(",").strip() for ele in abbrev_states]
fullname_states = [ele.strip("'").strip('"').strip(",").strip() for ele in fullname_states]

#the following states appear as abbreviations
abbrev_states

#the following states appear as full names
fullname_states

#make a list instead of a set to find value counts for abbreviated states
abbrev_states_list = re.findall(regex_abbrev, str(debar["City, State"].tolist()))
abbrev_states_list = [ele.strip("'").strip('"').strip(",").strip() for ele in abbrev_states_list]
abbrev_states_list = pd.Series(abbrev_states_list)
abbrev_states_list.value_counts()

#make a list instead of a set to find value counts for full name states
fullname_states_list = re.findall(regex_full, str(debar["City, State"].tolist()))
fullname_states_list = [ele.strip("'").strip('"').strip(",").strip() for ele in fullname_states_list]
fullname_states_list = pd.Series(fullname_states_list)
fullname_states_list.value_counts()

print("In addition to all the states with value counts over 1, Idaho has a value count of 1 in both lists,\nso it also qualifies as having multiple violations.")

['CO',
 'FL',
 'TN',
 'AR',
 'KS',
 'VT',
 'ID',
 'SC',
 'UT',
 'LA',
 'GA',
 'MA',
 'CA',
 'SD',
 'AK',
 'ME',
 'OK',
 'IL',
 'MN',
 'MD',
 'ND',
 'MT',
 'TX',
 'NY',
 'KY']

[]

GA    19
TX    18
FL    10
KS    10
UT     6
KY     5
SD     5
CA     4
CO     3
NY     3
ND     3
MA     3
LA     3
AR     3
ID     2
MT     2
OK     2
SC     1
ME     1
MN     1
TN     1
MD     1
AK     1
VT     1
IL     1
dtype: int64

/var/folders/0b/sg22rdx51yg9xnh9k5v8n2qm0000gn/T/ipykernel_23428/3206589978.py:30: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  fullname_states_list = pd.Series(fullname_states_list)


Series([], dtype: int64)

In addition to all the states with value counts over 1, Idaho has a value count of 1 in both lists,
so it also qualifies as having multiple violations.


To make the state names consistent with the jobs data below, convert any discrepant state names to the two-letter abbreviation format. 

**Hints:**
- You could do this for all state names (to be safe) or only those you just identified. 
- If you want to change ANY discrepant state names from long format to two-letter format (i.e., a complete conversion), you can load state names from a complete name/abbreviation crosswalk and use that to change state names. Here is code to load in such a list ([from this blog](https://towardsdatascience.com/state-name-to-state-abbreviation-crosswalks-6936250976c)):
```python
crosswalk_url = 'http://app02.clerk.org/menu/ccis/Help/CCIS%20Codes/state_codes.html'
pd.read_html(crosswalk_url)[0]
```

In [436]:
#import conversion table
crosswalk_url = 'http://app02.clerk.org/menu/ccis/Help/CCIS%20Codes/state_codes.html'
statecodesdf = pd.read_html(crosswalk_url)[0]

## your code here

#loop over all the states whose full names are used
for ele in fullname_states:
    
    #replace the full name with the corresponding code
    debar = debar.replace(to_replace = ele, value = statecodesdf.loc[statecodesdf.Description == ele, "Code"].iloc[-1], regex = True)
    
debar.head()

,Name,"City, State",Violation,Duration,Start date,End date,violation_number,is_repeated
94,69 Farms LLC,"Preston, ID",Failure to respond to audit request,2 years,9/24/2016,9/23/2018,viol,False
97,AB Ranch,"Stephenville, TX",Failure to respond to audit request,2 years,4/28/2017,4/27/2019,viol,False
102,Abe- Nancy Froese,"Seminole, TX",Failure to respond to audit request,2 years,11/23/2017,11/23/2019,viol,False
36,Agecy I LLC,"Idabel, OK",Impeding the Audit Process – Non- Response,2 years,8/27/2015,8/26/2017,viol,False
53,"Agecy II, LLC","Ipswich, SD",Impeding the Audit Process – Non- Response,2 years,3/25/2015,3/24/2017,viol,False


## 1.3 Investigate duplicated rows (4 points)

A. Create a new column in `debar`, `is_repeated`, that tells us whether an employer (`Name`) is repeated > 1 times.

*Hint*: there are multiple ways to solve this but some possibilities to get the list of names that are repeated are:
- Using `value_counts()` on the `Name` variable and extracting the index from that value counts 
- Using groupby to count the rows attached to one name

B. Print the rows where `is_repeated == True` and interpret with at least a sentence. If you notice any cases of duplicate business names where 'City, State' does NOT match exactly, check these manually and make them consistent.

C. Subset to the rows where `is_repeated == True` and save that data as `mult_debar`. Print the head() and shape

In [437]:
## your code here


#create new column for repeat names
debar["is_repeated"] = [False if debar["Name"].tolist().count(ele) == 1
                       else True
                       for ele in debar["Name"]]



#check the repeat names and interpret
debar.loc[debar.is_repeated == True]
print("The following names appear multiple times. Most of them appear to have been duplicate entries, like Annabella Land & Cattle, with only slightly differently worded violations and start/end dates. Xavier Horne appears to be two discrete violations. Loewen Harvesting is recorded as existing in Brownfield and Brownsville, TX; I looked it up and determined that Brownsville was a typo. Maple Ridge Custom Services is also listed in Alaska and Arkansas; I've determined it's located in Arkansas.")



#fix Brownfield and Altheimer errors
debar = debar.replace(to_replace = "Brownsville", value = "Brownfield", regex = True)
debar = debar.replace(to_replace = "mer, AS", value = "mer, AR", regex = True)



#create mult_debar dataframe
mult_debar = debar[debar["is_repeated"] == True]


#find head and shape
mult_debar.head()
mult_debar.shape


,Name,"City, State",Violation,Duration,Start date,End date,violation_number,is_repeated
30,Annabella Land & Cattle,"Annabella, UT",Non-payment,1 year,5/9/2014,5/8/2015,viol,True
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,viol2,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,viol,True
29,Autumn Hill Orchard,"Groton, MA",Impeding the Audit Process – Non- Response,2 years,7/6/2014,7/5/2016,viol2,True
28,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Impeding the Audit Process – Partial- Response,2 years,7/20/2014,7/19/2016,viol,True
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,viol2,True
19,Cisco Produce Inc.,"Cairo, GA",Failure to respond to audit (no response),2 years,12/10/2014,12/9/2016,viol,True
56,Cisco Produce Inc.,"Cairo, GA",Impeding the Audit Process – Non- Response,2 years,12/10/2015,12/9/2017,viol2,True
109,Dove Creek Farms,"Mount Vernon, TX",Failure to Respond to Audit Request,2 years,2/9/2018,2/8/2020,viol,True
103,Dove Creek Farms,"Mount Vernon, TX",Failure to respond to audit request,2 years,2/9/2018,2/8/2018,viol2,True


The following names appear multiple times. Most of them appear to have been duplicate entries, like Annabella Land & Cattle, with only slightly differently worded violations and start/end dates. Xavier Horne appears to be two discrete violations. Loewen Harvesting is recorded as existing in Brownfield and Brownsville, TX; I looked it up and determined that Brownsville was a typo. Maple Ridge Custom Services is also listed in Alaska and Arkansas; I've determined it's located in Arkansas.


,Name,"City, State",Violation,Duration,Start date,End date,violation_number,is_repeated
30,Annabella Land & Cattle,"Annabella, UT",Non-payment,1 year,5/9/2014,5/8/2015,viol,True
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,viol2,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,viol,True
29,Autumn Hill Orchard,"Groton, MA",Impeding the Audit Process – Non- Response,2 years,7/6/2014,7/5/2016,viol2,True
28,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Impeding the Audit Process – Partial- Response,2 years,7/20/2014,7/19/2016,viol,True


(32, 8)

## 1.4 Reshape `mult_debar` to wide format to begin filtering out duplicates (4 points)

You want to separate out two cases:

- Cases where the repeat rows for one employer are due to duplicated data 
- Cases where the repeat rows for one employer represent repeated violations for different issues

There are various ways to check duplicates in these data: e.g., converting `Violation` to lowercase or replacing spelled-out states with two-dig state codes.

We're going to use the simple rule of:

- A row is a duplicate if, within an employer (defined by Name + City, State), the Start date for each row's violation is the same 

To begin to check this, reshape `mult_debar` to a wide dataframe (`mult_debar_wide`) with the following columns, treating the `Name` and `City, State` as the index for the pivot:

- Name
- City, State
- start_date_viol1
- start_date_viol2

For a successful reshaping, make sure each row in `mult_debar` shows only a single business (no duplicates under `Name`) and isn't missing either `start_date_viol1` or `start_date_viol2`.

Show the contents of `mult_debar` and its shape.

In [438]:
## your code here
#Create the pivot table
mult_debar_wide = pd.pivot(mult_debar, index = ["Name", "City, State"],
                                columns = ["violation_number"],
                                values = ["Start date"]).reset_index()

#rename columns
mult_debar_wide.columns = ["Name", "Location", "start_date_viol1", "start_date_viol2"]
             
mult_debar_wide

,Name,Location,start_date_viol1,start_date_viol2
0,Annabella Land & Cattle,"Annabella, UT",5/9/2014,5/9/2014
1,Autumn Hill Orchard,"Groton, MA",7/6/2014,7/6/2014
2,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",7/20/2014,7/20/2014
3,Cisco Produce Inc.,"Cairo, GA",12/10/2014,12/10/2015
4,Dove Creek Farms,"Mount Vernon, TX",2/9/2018,2/9/2018
5,F&W Farms,"Ingalls, KS",12/10/2014,12/10/2014
6,Loewen Harvesting LLC,"Brownfield, TX",8/20/2014,8/20/2014
7,Macky and Brad Farms,"Plains, TX",2/13/2015,2/13/2015
8,"Maple Ridge Custom Services, LLC","Altheimer, AK",NaN,11/16/2014
9,"Maple Ridge Custom Services, LLC","Altheimer, AR",11/16/2014,NaN


## 1.5 Filter out duplicates from original debar data (6 points)

A. Using `mult_debar_wide`, add a column `is_dup` that takes value of True for cases where start_date_viol1 == start_date_viol2 marking the row as a duplicate

B. Going back to the original long-format data you loaded at the beginning, `debar`, filter as follows:
- For employers where `is_dup == True` as indicated by your wide-format dataframe, only keep `violnum == viol1`
- For all other employers (so is_dup == False and ones we didnt need to check duplicates for), keep all violnum
- Remove the `is_repeated` column from the `debar` data

**Hint**: you can complete part B without a for loop; `pd.concat` with axis = 0 (row binding) is one way

Call the resulting dataframe `debar_clean` and print the shape and # of unique employer names

In [439]:
## your code here

#create is_dup column
mult_debar_wide["is_dup"] = [x == y for x, y in zip(mult_debar_wide["start_date_viol1"], mult_debar_wide["start_date_viol2"])]
mult_debar_wide

#create a df of repeat violators
debar_repeats = debar.loc[debar.violation_number == "viol2"]

#create a list of duplicate names
dupes = mult_debar_wide.loc[mult_debar_wide["is_dup"] == True].Name.tolist()

#remove duplicate names from df of repeat violators
debar_repeats = debar_repeats.loc[~debar_repeats["Name"].isin(dupes)]

#create new dataframe with the first violations and the "true" repeats
debar_clean = pd.concat((debar.loc[debar.violation_number == "viol"], debar_repeats), axis = 0)
debar_clean






,Name,Location,start_date_viol1,start_date_viol2,is_dup
0,Annabella Land & Cattle,"Annabella, UT",5/9/2014,5/9/2014,True
1,Autumn Hill Orchard,"Groton, MA",7/6/2014,7/6/2014,True
2,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",7/20/2014,7/20/2014,True
3,Cisco Produce Inc.,"Cairo, GA",12/10/2014,12/10/2015,False
4,Dove Creek Farms,"Mount Vernon, TX",2/9/2018,2/9/2018,True
5,F&W Farms,"Ingalls, KS",12/10/2014,12/10/2014,True
6,Loewen Harvesting LLC,"Brownfield, TX",8/20/2014,8/20/2014,True
7,Macky and Brad Farms,"Plains, TX",2/13/2015,2/13/2015,True
8,"Maple Ridge Custom Services, LLC","Altheimer, AK",NaN,11/16/2014,False
9,"Maple Ridge Custom Services, LLC","Altheimer, AR",11/16/2014,NaN,False


,Name,"City, State",Violation,Duration,Start date,End date,violation_number,is_repeated
94,69 Farms LLC,"Preston, ID",Failure to respond to audit request,2 years,9/24/2016,9/23/2018,viol,False
97,AB Ranch,"Stephenville, TX",Failure to respond to audit request,2 years,4/28/2017,4/27/2019,viol,False
102,Abe- Nancy Froese,"Seminole, TX",Failure to respond to audit request,2 years,11/23/2017,11/23/2019,viol,False
36,Agecy I LLC,"Idabel, OK",Impeding the Audit Process – Non- Response,2 years,8/27/2015,8/26/2017,viol,False
53,"Agecy II, LLC","Ipswich, SD",Impeding the Audit Process – Non- Response,2 years,3/25/2015,3/24/2017,viol,False
...,...,...,...,...,...,...,...,...
27,Yolanda Chavez Farming,"Santa Maria, CA",Non-Payment,1 year,7/23/2014,7/22/2015,viol,False
56,Cisco Produce Inc.,"Cairo, GA",Impeding the Audit Process – Non- Response,2 years,12/10/2015,12/9/2017,viol2,True
17,"Maple Ridge Custom Services, LLC","Altheimer, AK",Failure to respond to audit (partial response),2 years,11/16/2014,11/15/2016,viol2,True
51,Old Tree Farms/Verpaalen Custom Service,"Volga, SD",Wage Hour Debarment,3 years,12/1/2014,12/1/2017,viol2,True


# 2. Merging and regex (19 points total)



## 2.1 Load data on job postings (1 point)

The previous dataset contains a small subset of employers who faced temporary bans due to violations of H-2A program regulations. Since most of the bans have expired, let's see which of those employers posted new H-2A jobs in the first quarter of 2021.

First, load the `jobs.csv` data stored in `pset3_inputdata`.

In [440]:
# your code here to load the data 

with ZipFile("pset3_inputdata.zip") as myzip:
    data = myzip.open("jobs.csv")

#read csv
jobs = pd.read_csv(data)

jobs.head()

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,ADDENDUM_B_HOUSING_ATTACHED,TOTAL_HOUSING_RECORDS,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS
0,H-300-20199-721302,Determination Issued - Withdrawn,2020-07-17 14:50:40.840,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,Y,"Fazio Farms Operating Company, LLC",NaN,...,N,1,Y,12.68,12.68,55.0,13607017661,faziofarms@gmail.com,NaN,0
1,H-300-20231-773906,Determination Issued - Certification,2020-08-20 10:38:15.620,2020-10-01 00:00:00.000,Association - Agent,N,Seasonal,N,Charlie Sunderland,Panter & Sunderland Nursery,...,N,1,N,NaN,12.68,55.0,19318083783,NaN,https://www.jobs4tn.gov/vosnet/Default.aspx,0
2,H-300-20231-774123,Determination Issued - Certification,2020-08-24 15:33:14.340,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,Michael Rudebusch,NaN,...,N,1,N,NaN,12.68,55.0,19369333827,fayethlynpitre@rocketmail.com,NaN,0
3,H-300-20231-774151,Determination Issued - Certification,2020-08-21 12:08:09.760,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,Lodahl Farms,NaN,...,Y,2,N,NaN,12.68,55.0,14069637560,lodahl_kelsey@yahoo.com,NaN,0
4,H-300-20231-774508,Determination Issued - Certification,2020-08-20 10:17:34.530,2020-10-01 00:00:00.000,Individual Employer,Y,Seasonal,N,"Dunson Harvesting, Inc.","Dunson Harvesting, Inc.",...,Y,8,N,NaN,12.68,55.0,18632939888,NaN,www.employflorida.com,4


The most notable column names mean as follows:
- `CASE_NUMBER`:	Administrative identifier for an employer's H-2A visa application
- `EMPLOYER_NAME`:	Employer name
- `EMPLOYER_CITY`:	Employer city
- `EMPLOYER_STATE`:	Employer state
- `EMPLOYER_ADDRESS_1`:	Employer address (only need to use if doing the geocoding extra credit)

##  2.2 Try inner join on employer name  (2 points)

- Use the `EMPLOYER_NAME` field of the `jobs` dataset
- Use the `Name` field of the `debar_clean` dataset 

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are exact matches, print the row(s) with exact matches



In [441]:
## your code here

#merge
merged_employers = pd.merge(jobs, debar_clean, left_on='EMPLOYER_NAME', right_on='Name', how = "inner")

#print row                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
merged_employers

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS,Name,"City, State",Violation,Duration,Start date,End date,violation_number,is_repeated
0,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,Rafael Barajas,NaN,...,https://seasonaljobs.dol.gov/,7,Rafael Barajas,"Sebring, FL",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,viol,False


## 2.3 Targeted regex (11 points total)

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the employer name fields in each dataset 

### 2.3.1 Convert to upper (3 points)

A. Convert the `EMPLOYER_NAME` and `Name` fields to uppercase using list comprehension rather than df.varname.str.upper() (it's fine to do a separate list comprehension line for each of the two columns)

B. Print a random sample of 15 values of each result

C. Assign the full vector of uppercase names back to the original data, writing over the original `EMPLOYER_NAME` and `Name` columns 


In [442]:
## insert your code to turn into uppercase here
from random import sample

capital_EMPLOYER_NAME = [ele.upper() for ele in jobs["EMPLOYER_NAME"]]
capital_Name = [ele.upper() for ele in debar_clean["Name"]]

In [443]:
## insert your code for the random sample

sample(capital_EMPLOYER_NAME, 15)

sample(capital_Name, 15)

['HOLDMAN HONEY, INC.',
 'VAN ZYVERDEN, INC. ',
 'AGRINATURAL GROWER SUPPLY ',
 'JERRY E JONES III FARMS',
 'M.A. GRAIN LLC',
 'KYLE R. FONTENOT',
 'FOOTHILL PACKING, INC.',
 'LIBERTY LAWN AND LANDSCAPING, INC. ',
 'CUSTOM ORCHARDS, INC.',
 'CORNELIU PRELIPCEANU',
 'BENJAMIN AND MICHELLE DESHOTELS FARMS',
 'VG FARM VENTURES',
 'HBW FARMS, INC.',
 'MOON VALLEY NURSERY, INC.',
 'STEPHEN TOWE']

['ROSALVA GARCIA',
 'LANDMARK LANDSCAPING',
 'MELVIN DON UNDERWOOD II',
 'TRIANGLE J FARMS',
 'GREAT PLAINS FLUID SERVICE, INC.',
 'LEONARD SMITH FARMS',
 'OMEGA LAMB, LLC',
 'JEREMY CHANDLER',
 'PETER PETERS',
 'ROBERT D. TOWLES',
 'STAHLMAN APIARIES, INC',
 'K W ENTERPRISES',
 'OLD TREE FARMS/VERPAALEN CUSTOM SERVICE',
 'FIRST AMERICAN HOLDING',
 'RAFAEL BARAJAS']

In [444]:
## insert your code for assigning the uppercase names back to the data
jobs["EMPLOYER_NAME"] = capital_EMPLOYER_NAME 
debar_clean["Name"] = capital_Name

### 2.3.2 Clean up punctuation (4 points)

You notice that INC, CO, and LLC are sometimes followed by a period (.) but sometimes not

A. For each dataset, write a regex pattern using `re.sub` to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example we provide below and print the result. See the Github issue for examples of what to return


**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content



In [445]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [446]:
## insert your code here with the regex pattern for part A
regex = r"(\s\w\w\w?)\."

## insert your code to use re.sub to apply the pattern to the test cases for part B
re.sub(regex, r"\1", pos_example_1)
re.sub(regex, r"\1", pos_example_2)
re.sub(regex, r"\1", neg_example)

'CISCO PRODUCE INC'

'AVOYELLES HONEY CO, LLC'

'E.V. RANCH LLP'

### 2.3.3 Clean employer names (4 points)

Use that pattern in conjunction with `re.sub` and list comprehension to clean the employer name columns in each dataset. Save the new columns as `name_clean` in each. Then, use row subsetting to (1) subset to rows that changed names and (2) for:

- `debar_clean` print the `Name` and `name_clean` columns
- `jobs` print the `EMPLOYER_NAME` and `name_clean` columns

Make sure to use the uppercase versions of the variables


In [447]:
## your code here to clean the columns
jobs["name_clean"] = [re.sub(regex, r"\1", ele) for ele in jobs["EMPLOYER_NAME"]]
debar_clean["name_clean"] = [re.sub(regex, r"\1", ele) for ele in debar_clean["Name"]]

#subset to rows that changed names
jobs_changed = jobs.loc[~jobs["name_clean"].isin(jobs["EMPLOYER_NAME"])]
debar_clean_changed = debar_clean.loc[~debar_clean["name_clean"].isin(debar_clean["Name"])]

In [448]:
## your code here to print the head
debar_clean_changed[["Name", "name_clean"]].head()
jobs_changed[["EMPLOYER_NAME", "name_clean"]].head()



,Name,name_clean
46,ALTENDORF TRANSPORT INC.,ALTENDORF TRANSPORT INC
3,ANTON FERTILIZER INC.,ANTON FERTILIZER INC
91,"AVOYELLES HONEY CO., LLC","AVOYELLES HONEY CO, LLC"
19,CISCO PRODUCE INC.,CISCO PRODUCE INC
33,GERONIMO SHEEP CO.,GERONIMO SHEEP CO


,EMPLOYER_NAME,name_clean
4,"DUNSON HARVESTING, INC.","DUNSON HARVESTING, INC"
7,"FARM LABOR ASSOCIATION FOR GROWERS, INC.","FARM LABOR ASSOCIATION FOR GROWERS, INC"
14,"MCLAIN FARMS, INC.","MCLAIN FARMS, INC"
17,"BONNIE PLANTS, INC.","BONNIE PLANTS, INC"
18,"B & W QUALITY GROWERS, INC.","B & W QUALITY GROWERS, INC"


## 2.4 More joins and more cleaning (5 points)

A. Conduct another inner join between `jobs` and `debar_clean` now using the `name_clean` column; print the result. Did the cleaning result in any more employers matched between the two datasets?

B. Create a new column in `debar_clean` called `name_clean_2` that uses regex to take the following name in that dataset:

- `SLASH E.V. RANCH LLP` in the `debar_clean` dataset

And cleans it up so that it matches with this employer in `jobs`

- `SLASH EV RANCH` in the `jobs` dataset

Eg a pattern to remove the dots in the EV and the space+LLP-- you can apply the pattern to all employer names in debar_clean (so don't need to worry about only applying it to that one employer)


C. Conduct a left join using `name_clean_2` as the join column where the left hand dataframe is `jobs`; right hand dataframe is `debar_clean`, store the result as a dataframe, and print the rows where the merge indicator indicates the row was found in both dataframe. Write a sentence describing how name cleaning affected the match results.

**Note**: this manual cleaning process is inefficient and may miss other likely matches. A better approach would be fuzzy matching, which would recognize that 'Slash EV ranch' is a highly similar string to 'slash ev ranch llp' and match them without us needing to use regex to make the strings identical. We may talk about this in class later if time!

In [466]:
## your code here

merged_employers2 = pd.merge(jobs, debar_clean, how = "inner")
merged_employers2

regex = r"\.(\w)\.(\sRANCH)\sLLP"
debar_clean["name_clean_2"] = [re.sub(regex, r"\1\2", ele) for ele in debar_clean["name_clean"]]



merged_employers3 = pd.merge(jobs, debar_clean, left_on = "name_clean", right_on = "name_clean_2", how = "left")
merged_employers3_matching = merged_employers3.loc[merged_employers3.name_clean_2.isnull() == False]
merged_employers3_matching


,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,name_clean,Name,"City, State",Violation,Duration,Start date,End date,violation_number,is_repeated,name_clean_2
0,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,RAFAEL BARAJAS,NaN,...,RAFAEL BARAJAS,RAFAEL BARAJAS,"Sebring, FL",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,viol,False,RAFAEL BARAJAS


,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,Name,"City, State",Violation,Duration,Start date,End date,violation_number,is_repeated,name_clean_y,name_clean_2
791,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,RAFAEL BARAJAS,NaN,...,RAFAEL BARAJAS,"Sebring, FL",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,viol,False,RAFAEL BARAJAS,RAFAEL BARAJAS
1115,H-300-20306-894148,Determination Issued - Certification,2020-11-02 18:11:29.140,2020-11-24 00:00:00.000,Individual Employer,N,Seasonal,N,SLASH EV RANCH,NaN,...,SLASH E.V. RANCH LLP,"Rifle, CO",WHD Debarment,1 year,11/15/2014,11/14/2015,viol,False,SLASH E.V. RANCH LLP,SLASH EV RANCH


The name cleaning didn't work at all, despite all the cleaning instructions appearing to have worked, since our only two matches are the one that matched already and the one that we manually forced to match.

# 3. Regex to separate companies from individuals (6 points)

You notice some employers in `debar_clean` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned `name_clean` in `debar_clean`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string; so in example above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the "and")
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
**Hints and resources**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

- Same regex resources as above
    

In [476]:
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"
neg_example = "CISCO PRODUCE INC"

## your code here to define the pattern
regex = r"(.+?)\s\(\w+?\)\sAND\s(.+?)\s\(\w+?\)"

## your code here to apply it to the pos_example
list(re.findall(regex, pos_example)[0])

## your code here to apply it to the negative example
re.findall(regex, neg_example)

['COUNTY FAIR FARM', 'ANDREW WILLIAMSON']

[]

C. Iterate over the `name_clean` column in debar and use regex to create two new columns in `debar_clean`:
   - `co_name`: A column for company (full `name_clean` string if no match; pattern before COMPANY if one extracted)
   - `ind_name`: A column for individual (full `name_clean` string if no match; pattern before INDIVIDUAL if one extracted)
 


In [493]:
# your code here
regex_coname = r"(.+?)\s\(C"
regex_indname = r"AND\s(.+?)\s\(I"

debar_clean["co_name"] = ["".join(re.findall(regex_coname, ele)) if re.findall(regex_coname, ele) != []
                          else ele
                          for ele in debar_clean["name_clean"]]
debar_clean["ind_name"] = ["".join(re.findall(regex_indname, ele)) if re.findall(regex_indname, ele) != []
                           else ele
                           for ele in debar_clean["name_clean"]]


   
D. Print three columns for the rows in `debar_clean` containing the negative example and positive example described above (county fair farm and cisco produce):

- `name_clean`
- `co_name`
- `ind_name`
- `Violation`

**Note**: as shown in the outcome there may be duplicates of the same company reflecting different violations

In [497]:
# your code here
pos_row = debar_clean.loc[debar_clean.name_clean == pos_example]
neg_row = debar_clean.loc[debar_clean.name_clean == neg_example]

pos_row[["name_clean", "co_name", "ind_name", "Violation"]]
neg_row[["name_clean", "co_name", "ind_name", "Violation"]]

,name_clean,co_name,ind_name,Violation
108,COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...,COUNTY FAIR FARM,ANDREW WILLIAMSON,WHD Debarment


,name_clean,co_name,ind_name,Violation
19,CISCO PRODUCE INC,CISCO PRODUCE INC,CISCO PRODUCE INC,Failure to respond to audit (no response)
56,CISCO PRODUCE INC,CISCO PRODUCE INC,CISCO PRODUCE INC,Impeding the Audit Process – Non- Response


# 4. Optional extra credit: Geospatial visualization (2 points)

Geocode the employer addresses in `jobs` and plot the addresses of jobs as points overlaid on a map of Georgia. This involves Googling and using external sources to figure out the code (a common practice in real-life data science), since we haven't spatial data in the course. 

**Hints:**
- Relevant columns include `EMPLOYER_ADDRESS_1` 
- The geocoding might have a long runtime, so feel free to implement it in a separate `.py` script that you submit alongside your notebook and to just read in the geocoded data

**Resources:**
- [Discussion of geocoding addresses -> lat/long](https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/)
- [Discussion of plotting lat/long dots against a map using geopandas](https://towardsdatascience.com/plotting-maps-with-geopandas-428c97295a73)

In [ ]:
## your code here